In [1]:
%%capture

#enable the process bar
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer
!pip install s3fs
!pip install boto3

print('everything installed')

In [1]:
# %%capture
# !pip install datasets==1.13.3
# !pip install transformers==4.11.3

UsageError: %%capture is a cell magic, but the cell body is empty.


In [2]:
import boto3
import json

In [3]:
from datasets import load_dataset, load_metric, Audio


In [5]:
# import keys from  local 


In [6]:
s3resource = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'] )

In [ ]:
# s3client = boto3.client(
# service_name='s3',
# region_name='eu-west-1',
# aws_access_key_id= data['aws_access_key_id'],
# aws_secret_access_key= data['aws_secret_access_key'] )

In [7]:
BUCKET = 'commonvoicesdataset'
FILE_TO_READ = 'Turkish/vocab.json'
object1 = s3resource.Object(BUCKET,FILE_TO_READ)
file_content = object1.get()['Body'].read().decode('utf-8')
A =json.loads(file_content)

In [8]:
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(A, vocab_file)

In [7]:
from datasets import load_from_disk

In [9]:
from datasets.filesystems import S3FileSystem
s3fsystems = S3FileSystem(key=data['aws_access_key_id'], secret=data['aws_secret_access_key'])  

In [10]:
train_set_loaded = load_from_disk('s3://commonvoicesdataset/Turkish/train_set',fs=s3fsystems)  

In [11]:
test_set_loaded = load_from_disk('s3://commonvoicesdataset/Turkish/test_set',fs=s3fsystems)  

In [12]:
train_set_loaded

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 3478
})

In [13]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer('./vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [14]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [15]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [17]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [18]:
import numpy  as np
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)



In [19]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [20]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", #facebook/wav2vec2-xls-r-300m
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_q.bias', 'project_hid.bias', 'project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [21]:
model.freeze_feature_extractor()

In [20]:
train_set

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./Turkish3",
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps", 
  gradient_checkpointing=True,
  num_train_epochs=1,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_set_loaded,
    eval_dataset=test_set_loaded,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [24]:
!nvidia-smi

Wed Feb 23 07:49:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0    26W /  70W |   2126MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [57]:
train_set

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [25]:
trainer.train()

***** Running training *****
  Num examples = 3478
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 435
/home/ec2-user/anaconda3/envs/s2t/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811756765/work/torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/home/ec2-user/anaconda3/envs/s2t/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorre

Step,Training Loss,Validation Loss,Wer
400,4.912000,3.110982,1.000000


***** Running Evaluation *****
  Num examples = 1647
  Batch size = 8
Saving model checkpoint to ./Turkish3/checkpoint-400
Configuration saved in ./Turkish3/checkpoint-400/config.json
Model weights saved in ./Turkish3/checkpoint-400/pytorch_model.bin
Configuration saved in ./Turkish3/checkpoint-400/preprocessor_config.json
/home/ec2-user/anaconda3/envs/s2t/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=435, training_loss=4.763171965500404, metrics={'train_runtime': 664.8999, 'train_samples_per_second': 5.231, 'train_steps_per_second': 0.654, 'total_flos': 4.2519016319514624e+17, 'train_loss': 4.763171965500404, 'epoch': 1.0})

In [28]:
%cd /home/ec2-user/SageMaker/STT/savedmodel

[Errno 2] No such file or directory: '/home/ec2-user/SageMaker/STT/savedmodel'
/home/ec2-user/SageMaker/STT


In [26]:
trainer.save_model('/home/ec2-user/SageMaker/STT/savedmodel')

Saving model checkpoint to /home/ec2-user/SageMaker/STT/savedmodel
Configuration saved in /home/ec2-user/SageMaker/STT/savedmodel/config.json
Model weights saved in /home/ec2-user/SageMaker/STT/savedmodel/pytorch_model.bin
Configuration saved in /home/ec2-user/SageMaker/STT/savedmodel/preprocessor_config.json


In [40]:
tokenizer.sa

/home/ec2-user/SageMaker


In [41]:
%rm -rf dataset

In [30]:
trainer.model.save_pretrained('/home/ec2-user/SageMaker/STT/savedmodel')

Configuration saved in /home/ec2-user/SageMaker/STT/savedmodel/config.json
Model weights saved in /home/ec2-user/SageMaker/STT/savedmodel/pytorch_model.bin


In [31]:
 %cd /home/ec2-user/SageMaker/STT/savedmodel/

/home/ec2-user/SageMaker/STT/savedmodel


In [32]:
%ls

config.json  pytorch_model.bin
